# Observable 

When making a GUI, the system must react only when something is changed (eg. user clicks somewhere). 

This class allows a Signal to be sent from an Observable which allows data to be modified

In [ ]:
from typing import Callable, Optional, Set, TypeVar, Generic


class Signal:
    """
    Signals are responsible for connecting obesrvables and observers, passing data from one to the other.
    
    Signal.connect() registers an observer function.
    Signal.send(args, kwargs) calls all registered observer functions with the args and kwargs.
    """
    def __init__(self) -> None:
        self._funs: Set[Callable] = set()

    def connect(self, fun) -> None:
        self._funs.add(fun)

    def send(self, *args, **kwargs) -> None:
        for fun in self._funs:
            fun(*args, **kwargs)


T = TypeVar('T')


class Observable(Generic[T]):
    """
    The Observable sends an "updated" Signal whenever its data attribute is updated.
    It's useful for managing state of immutable data.
    """
    updated: Signal

    def __init__(self, data: T, updated: Optional[Signal] = None):
        self._data = data
        self.updated: Signal = updated if updated else Signal()

    def send_all(self) -> None:
        self.updated.send(self._data)

    @property
    def data(self) -> T:
        return self._data
    
    @data.setter
    def data(self, value: T) -> None:
        self._data = value
        self.updated.send(self._data)

## Unit tests

In [ ]:

from unittest.mock import Mock
from web.observable import Signal, Observable


def test_signal_doesnt_call_functions_on_connection():
    signal = Signal()
    fun1 = Mock()

    signal.connect(fun1)
    
    fun1.assert_not_called()


def test_signal_pipes_data_to_all_connected_functions():
    signal = Signal()
    fun1 = Mock()
    fun2 = Mock()

    signal.connect(fun1)
    
    signal.send(hi='hello')
    fun1.assert_called_once_with(hi='hello')
    fun2.assert_not_called()

    signal.connect(fun2)
    signal.send(bye='goodbye')
    fun1.assert_called_with(bye='goodbye')
    fun2.assert_called_once_with(bye='goodbye')

    
    

def test_state_sends_update_whenever_a_new_model_is_set():
    update_signal = Mock()
    state = Observable(data=Mock(), updated=update_signal)
    update_signal.send.assert_not_called()

    for _ in range(3):
        new_model = Mock()
        state.data = new_model
        update_signal.send.assert_called_with(new_model)


def tests_state_sendall_sends_update():
    update_signal = Mock()
    data = Mock()
    state = Observable(data=data, updated=update_signal)

    update_signal.send.assert_not_called()
    state.send_all()
    update_signal.send.assert_called_with(data)

        